### Jorge Diego Garcia Torres

Ensamble de modelos

In [ ]:
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn import ensemble
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

# Primer modelo

In [ ]:
gender = {'F':0, 'M':1}
age = {'0-17':0, '18-25':1, '26-35':2, '36-45':3, '46-50':4, '51-55':5, '55+':6}
city = {'A':0, 'B':1, 'C':2}
staycity = {'0':0, '1':1, '2':2, '3':3, '4+':4}

In [ ]:
def XGB(X_train, y_train, X_test):
  hyperparams = {}
  hyperparams["objective"] = "reg:linear"
  hyperparams["eta"] = 0.05
  hyperparams["min_child_weight"] = 10
  hyperparams["subsample"] = 0.8
  hyperparams["colsample_bytree"] = 0.7
  hyperparams["silent"] = 1
  hyperparams["max_depth"] = 10
        #params["max_delta_step"]=2
  hyperparams["seed"] = 0
        #params['eval_metric'] = "auc"
  hyplist = list(hyperparams.items())
  epochs = 750
  xgb_train = xgb.DMatrix(X_train, label=y_train)
  xgb_test = xgb.DMatrix(X_test)
  model = xgb.train(hyplist, xgb_train, epochs)
  pred_y = model.predict(xgb_test)
  return pred_y

In [ ]:
def getSumofAtr(maindf,maindf2,atrname):
  groupdf = maindf2.groupby(atrname)
  count = {}
  print(groupdf)
  for name, category in groupdf:
    count[name] = category.shape[0]
  countl = []
  print(count)
  for index, row in maindf.iterrows():
    name = row[atrname]
    countl.append(count.get(name,0))
  print(countl)
  return countl


In [ ]:
def getAcquireAtr(maindf,maindf2,atrname):
  groupdf = maindf2.groupby(atrname)
  mind = {}
  maxd = {}
  meand = {}
  for name, cat in groupdf:
    mind[name] = min(np.array(cat["Purchase"]))
    maxd[name] = max(np.array(cat["Purchase"]))
    meand[name] = np.mean(np.array(cat["Purchase"]))
  
  minl = []
  maxl = []
  meanl = []
  for i, r in maindf.iterrows():
    name = r[atrname]
    minl.append(mind.get(name,0))
    maxl.append(maxd.get(name,0))
    meanl.append(meand.get(name,0))

  return minl,maxl,meanl



In [ ]:
maindf = pd.read_csv('txn_train.csv')

In [ ]:
maindf

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1004122,P00238542,M,18-25,19,C,4+,0,5,14.0,NaN,3522
1,1002348,P00111142,M,51-55,12,C,4+,1,1,15.0,16.0,11654
2,1002931,P00042842,M,18-25,4,B,2,0,5,NaN,NaN,3456
3,1003808,P00195342,M,26-35,7,A,2,1,2,5.0,NaN,13288
4,1004950,P0099242,M,26-35,16,A,1,1,8,14.0,NaN,5915
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1001764,P00015542,M,18-25,0,B,1,0,1,2.0,13.0,15656
49996,1005925,P00205942,F,26-35,0,C,1,1,5,8.0,14.0,6863
49997,1001511,P00265742,M,36-45,12,C,4+,1,5,12.0,NaN,7052
49998,1003224,P00037542,F,26-35,14,A,3,1,1,NaN,NaN,7866


In [ ]:
maindf["Gender"] = maindf["Gender"].apply(lambda x: gender[x])

In [ ]:
maindf["Age"] = maindf["Age"].apply(lambda x: age[x])

In [ ]:
maindf["City_Category"] = maindf["City_Category"].apply(lambda x: city[x])

In [ ]:
maindf["Stay_In_Current_City_Years"] = maindf["Stay_In_Current_City_Years"].apply(lambda x: staycity[x])

## Se pueden o no remover outliers antes de agregar conteos, maximos, minimos, poder adquisitivo de los productos y usuarios

In [ ]:
def outlier_removal(df):
  q75,q25 = np.percentile(df.loc[:,'Purchase'],[75,25])
  iqr = q75-q25
  min = q25 - (iqr*1.5)
  max = q75 + (iqr*1.5)
  df = df.drop(df[df.loc[:,'Purchase']<min].index)
  df = df.drop(df[df.loc[:,'Purchase']>max].index)
  return df

In [ ]:
maindf = outlier_removal(maindf)

In [ ]:
maindf.shape

(50000, 12)

- Se obtienen las totales de cada atributo
- El minimo, maximo y promedio de cada usuario, tipo poder adquisitivo
- El minimo, maximo y promedio de cada producto, tipo el costo 

In [ ]:
maindf["Agesum"] =  getSumofAtr(maindf, maindf, "Age")

{0: 1379, 1: 9051, 2: 19795, 3: 10078, 4: 4262, 5: 3521, 6: 1914}
[9051, 3521, 9051, 19795, 19795, 10078, 10078, 4262, 9051, 19795, 3521, 9051, 19795, 4262, 10078, 19795, 19795, 3521, 9051, 10078, 19795, 9051, 19795, 9051, 10078, 19795, 19795, 1914, 10078, 19795, 19795, 10078, 19795, 10078, 19795, 4262, 3521, 19795, 19795, 9051, 9051, 9051, 9051, 19795, 4262, 9051, 9051, 1379, 19795, 9051, 9051, 10078, 19795, 9051, 19795, 19795, 4262, 10078, 19795, 10078, 3521, 19795, 3521, 19795, 19795, 19795, 3521, 9051, 19795, 10078, 19795, 9051, 19795, 10078, 19795, 19795, 4262, 9051, 19795, 19795, 19795, 9051, 9051, 19795, 19795, 1914, 19795, 10078, 9051, 9051, 10078, 9051, 19795, 4262, 10078, 9051, 10078, 10078, 10078, 10078, 3521, 3521, 9051, 3521, 9051, 19795, 1914, 4262, 4262, 19795, 10078, 19795, 19795, 19795, 19795, 19795, 19795, 19795, 19795, 4262, 10078, 10078, 9051, 9051, 19795, 9051, 10078, 19795, 3521, 4262, 19795, 9051, 19795, 1914, 10078, 19795, 19795, 19795, 9051, 19795, 9051, 9051, 

In [ ]:
maindf["Occupationsum"] = getSumofAtr(maindf, maindf, "Occupation")

{0: 6341, 1: 4175, 2: 2398, 3: 1526, 4: 6561, 5: 1061, 6: 1907, 7: 5353, 8: 152, 9: 619, 10: 1186, 11: 1067, 12: 2805, 13: 685, 14: 2522, 15: 1139, 16: 2360, 17: 3697, 18: 600, 19: 765, 20: 3081}
[765, 2805, 6561, 5353, 2360, 4175, 6341, 3697, 6561, 3081, 4175, 3697, 2360, 3697, 6341, 2398, 1139, 4175, 6561, 2805, 2805, 6561, 1061, 6561, 6341, 2398, 6341, 1526, 5353, 2522, 4175, 1067, 1526, 2805, 1526, 1907, 6341, 1139, 6341, 6341, 6561, 2360, 6561, 6341, 5353, 6561, 6561, 1186, 765, 5353, 6561, 2360, 1526, 6561, 1907, 3081, 6341, 5353, 2805, 5353, 3697, 5353, 4175, 1907, 3081, 2805, 4175, 6561, 765, 4175, 5353, 1907, 4175, 2805, 2360, 2398, 1067, 6561, 6561, 5353, 2398, 6341, 6561, 4175, 5353, 685, 3081, 4175, 765, 6341, 2398, 6561, 5353, 6341, 1526, 6561, 619, 6341, 4175, 4175, 4175, 4175, 6561, 1067, 2522, 765, 5353, 4175, 4175, 3697, 5353, 6341, 6341, 6341, 5353, 3697, 2360, 3697, 3697, 4175, 5353, 4175, 6561, 6561, 6561, 6561, 2360, 4175, 3081, 6561, 2805, 600, 6561, 1526, 6341, 5

In [ ]:
maindf["Product_Category_1sum"] = getSumofAtr(maindf, maindf, "Product_Category_1")

{1: 12816, 2: 2190, 3: 1852, 4: 1082, 5: 13659, 6: 1965, 7: 331, 8: 10312, 9: 30, 10: 465, 11: 2186, 12: 352, 13: 491, 14: 139, 15: 591, 16: 876, 17: 55, 18: 264, 19: 145, 20: 199}
[13659, 12816, 13659, 2190, 10312, 10312, 1082, 12816, 12816, 1965, 10312, 1965, 2190, 12816, 12816, 12816, 1852, 13659, 12816, 13659, 1965, 1965, 13659, 13659, 13659, 10312, 13659, 145, 13659, 1852, 12816, 10312, 12816, 12816, 13659, 12816, 13659, 1965, 13659, 12816, 10312, 1082, 2190, 1965, 591, 1965, 12816, 1852, 12816, 13659, 1965, 12816, 591, 12816, 13659, 1082, 1965, 13659, 13659, 10312, 13659, 13659, 1965, 12816, 1852, 491, 13659, 12816, 491, 1852, 10312, 1852, 13659, 2186, 352, 12816, 13659, 13659, 10312, 12816, 10312, 12816, 12816, 12816, 12816, 1965, 10312, 10312, 10312, 491, 13659, 10312, 12816, 13659, 13659, 13659, 1965, 2186, 2190, 13659, 12816, 10312, 12816, 12816, 876, 10312, 10312, 12816, 264, 13659, 12816, 1082, 12816, 352, 2186, 465, 13659, 12816, 264, 10312, 10312, 12816, 199, 10312, 13659

In [ ]:
maindf["ProductCategory_2sum"] = getSumofAtr(maindf, maindf, "Product_Category_2")

{2.0: 4506, 3.0: 292, 4.0: 2339, 5.0: 2411, 6.0: 1523, 7.0: 56, 8.0: 5889, 9.0: 503, 10.0: 299, 11.0: 1209, 12.0: 514, 13.0: 955, 14.0: 4995, 15.0: 3434, 16.0: 3993, 17.0: 1201, 18.0: 224}
[4995, 3434, 0, 2411, 4995, 4995, 2411, 3434, 4995, 3993, 955, 3993, 2339, 2411, 1523, 4506, 2339, 1209, 1523, 4995, 0, 5889, 1209, 0, 0, 4995, 5889, 0, 0, 2339, 4506, 0, 2411, 3434, 4995, 3993, 4995, 5889, 4995, 224, 4995, 2411, 2339, 3993, 0, 5889, 0, 2339, 4506, 5889, 299, 4995, 3993, 4506, 5889, 2411, 5889, 5889, 0, 0, 0, 0, 5889, 5889, 2339, 0, 0, 3993, 3993, 2339, 514, 2339, 5889, 0, 4995, 3434, 4995, 5889, 0, 1523, 4995, 3993, 4506, 1209, 4506, 5889, 0, 0, 4995, 3993, 514, 0, 5889, 0, 0, 3993, 3993, 3434, 0, 0, 4995, 4995, 3434, 0, 0, 4995, 1201, 2411, 0, 5889, 4506, 2411, 3434, 4995, 4995, 955, 5889, 3434, 0, 0, 0, 4506, 0, 3993, 5889, 0, 0, 5889, 0, 2339, 4506, 503, 2411, 1201, 0, 0, 1201, 3434, 4995, 3993, 0, 955, 0, 4506, 0, 5889, 955, 0, 4506, 1201, 4995, 955, 5889, 4506, 5889, 3434, 0, 0

In [ ]:
maindf["ProductCategory_3sum"] = getSumofAtr(maindf, maindf, "Product_Category_3")

{3.0: 51, 4.0: 159, 5.0: 1552, 6.0: 461, 8.0: 1156, 9.0: 1091, 10.0: 182, 11.0: 142, 12.0: 836, 13.0: 498, 14.0: 1644, 15.0: 2547, 16.0: 3009, 17.0: 1515, 18.0: 397}
[0, 3009, 0, 0, 0, 0, 397, 0, 3009, 0, 0, 0, 1091, 836, 1644, 1091, 836, 0, 3009, 0, 0, 0, 0, 0, 0, 1515, 0, 0, 0, 1552, 2547, 0, 0, 0, 0, 0, 1515, 0, 0, 0, 0, 0, 1552, 0, 0, 3009, 0, 0, 0, 0, 498, 0, 0, 1091, 0, 1156, 3009, 1644, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1552, 0, 836, 0, 0, 0, 1515, 0, 0, 0, 3009, 0, 0, 2547, 2547, 1552, 3009, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3009, 0, 0, 0, 0, 0, 0, 397, 0, 0, 2547, 0, 0, 0, 0, 0, 0, 3009, 0, 0, 0, 0, 0, 0, 1644, 0, 0, 1644, 0, 0, 2547, 1644, 836, 0, 0, 0, 0, 0, 1515, 0, 0, 1644, 0, 1552, 0, 1515, 3009, 0, 2547, 0, 0, 0, 0, 1552, 0, 1515, 0, 0, 1552, 397, 0, 0, 0, 0, 0, 0, 0, 1552, 3009, 0, 498, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 461, 2547, 0, 0, 0, 3009, 0, 1644, 0, 182, 0, 3009, 0, 0, 0, 2547, 0, 0, 0, 0, 0, 1644, 0, 1552, 0, 0, 0, 0, 1552, 836, 2547, 0, 1552, 0, 1552, 0, 15

In [ ]:
maindf["UserIDsum"] = getSumofAtr(maindf, maindf, "User_ID")

{1000001: 2, 1000002: 4, 1000003: 4, 1000005: 16, 1000006: 2, 1000007: 1, 1000008: 7, 1000009: 3, 1000010: 22, 1000011: 5, 1000012: 1, 1000013: 5, 1000014: 1, 1000015: 5, 1000016: 3, 1000017: 8, 1000018: 13, 1000019: 9, 1000020: 1, 1000021: 1, 1000022: 20, 1000023: 12, 1000024: 10, 1000025: 1, 1000026: 23, 1000027: 5, 1000028: 7, 1000029: 4, 1000030: 5, 1000031: 5, 1000032: 1, 1000033: 20, 1000034: 16, 1000035: 13, 1000036: 20, 1000037: 6, 1000039: 4, 1000041: 2, 1000042: 9, 1000043: 2, 1000044: 17, 1000045: 19, 1000046: 4, 1000047: 1, 1000048: 38, 1000049: 2, 1000051: 1, 1000053: 38, 1000054: 2, 1000055: 2, 1000056: 4, 1000057: 3, 1000058: 25, 1000059: 18, 1000060: 3, 1000061: 1, 1000062: 23, 1000063: 6, 1000064: 1, 1000065: 5, 1000066: 1, 1000067: 2, 1000068: 3, 1000069: 2, 1000070: 6, 1000071: 3, 1000072: 2, 1000073: 7, 1000074: 3, 1000075: 15, 1000076: 6, 1000077: 1, 1000078: 6, 1000079: 1, 1000081: 2, 1000082: 5, 1000083: 1, 1000084: 2, 1000086: 2, 1000087: 4, 1000090: 17, 1000091

In [ ]:
maindf["ProductIDsum"] = getSumofAtr(maindf, maindf, "Product_ID")

{'P00000142': 100, 'P00000242': 44, 'P00000342': 17, 'P00000442': 8, 'P00000542': 14, 'P00000642': 47, 'P00000742': 22, 'P00000842': 1, 'P00000942': 9, 'P00001042': 53, 'P00001142': 62, 'P00001242': 11, 'P00001342': 2, 'P00001442': 8, 'P00001542': 8, 'P00001642': 31, 'P00001742': 28, 'P00001842': 13, 'P00001942': 22, 'P00002042': 6, 'P00002142': 68, 'P00002242': 20, 'P00002342': 6, 'P00002442': 19, 'P00002542': 43, 'P00002642': 7, 'P00002742': 6, 'P00002842': 14, 'P00002942': 12, 'P00003042': 6, 'P00003142': 9, 'P00003242': 81, 'P00003442': 77, 'P00003542': 1, 'P00003642': 52, 'P00003842': 2, 'P00003942': 66, 'P00004042': 1, 'P00004142': 10, 'P00004242': 13, 'P00004342': 8, 'P00004442': 9, 'P00004542': 19, 'P00004642': 6, 'P00004742': 67, 'P00004842': 23, 'P00004942': 2, 'P00005042': 97, 'P00005142': 19, 'P00005242': 1, 'P00005342': 1, 'P00005442': 2, 'P00005542': 1, 'P00005642': 4, 'P00005742': 23, 'P00005942': 15, 'P00006042': 4, 'P00006142': 16, 'P00006242': 14, 'P00006342': 3, 'P00

In [ ]:
min_price, max_price, mean_price = getAcquireAtr(maindf, maindf, "User_ID")

In [ ]:
maindf["UserIDMin"] = min_price
maindf["UserIDMax"] = max_price
maindf["UserIDMean"] = mean_price

In [ ]:
min_price, max_price, mean_price = getAcquireAtr(maindf, maindf, "Product_ID")
maindf["ProductIDMin"] = min_price
maindf["ProductIDMax"] = max_price
maindf["ProductIDMean"] = mean_price

Algunas categorias no se encuentran, en las estadisticas anteriores se puede observar que si se eliminan estos datos se perdiria el 30% o hasta el 60% de los datos, por lo tanto se puede crear una nueva categoria 0


In [ ]:
maindf.fillna(0, inplace=True)

Se codifican las variables categoricas en one hot

In [ ]:
categorical_columns = ["Gender","Age", "Occupation", "City_Category", "Stay_In_Current_City_Years",
                       "Marital_Status", "Product_Category_1", "Product_Category_2", "Product_Category_3"]

In [ ]:
maindf = pd.get_dummies(maindf, columns= categorical_columns)

In [ ]:
maindf.shape

(50000, 110)

In [ ]:
y_train = np.array(maindf["Purchase"])

In [ ]:
maindf = maindf.drop(columns =['User_ID','Product_ID','Purchase'])

In [ ]:
X = np.array(maindf).astype('float')

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
Scaler = StandardScaler()
Scaler.fit(X)
X = Scaler.transform(X)

In [ ]:
maindf

,Agesum,Occupationsum,Product_Category_1sum,ProductCategory_2sum,ProductCategory_3sum,UserIDsum,ProductIDsum,UserIDMin,UserIDMax,UserIDMean,ProductIDMin,ProductIDMax,ProductIDMean,Gender_0,Gender_1,Age_0,Age_1,Age_2,Age_3,Age_4,Age_5,Age_6,Occupation_0,Occupation_1,Occupation_2,Occupation_3,Occupation_4,Occupation_5,Occupation_6,Occupation_7,Occupation_8,Occupation_9,Occupation_10,Occupation_11,Occupation_12,Occupation_13,Occupation_14,Occupation_15,Occupation_16,Occupation_17,...,Product_Category_1_15,Product_Category_1_16,Product_Category_1_17,Product_Category_1_18,Product_Category_1_19,Product_Category_1_20,Product_Category_2_0.0,Product_Category_2_2.0,Product_Category_2_3.0,Product_Category_2_4.0,Product_Category_2_5.0,Product_Category_2_6.0,Product_Category_2_7.0,Product_Category_2_8.0,Product_Category_2_9.0,Product_Category_2_10.0,Product_Category_2_11.0,Product_Category_2_12.0,Product_Category_2_13.0,Product_Category_2_14.0,Product_Category_2_15.0,Product_Category_2_16.0,Product_Category_2_17.0,Product_Category_2_18.0,Product_Category_3_0.0,Product_Category_3_3.0,Product_Category_3_4.0,Product_Category_3_5.0,Product_Category_3_6.0,Product_Category_3_8.0,Product_Category_3_9.0,Product_Category_3_10.0,Product_Category_3_11.0,Product_Category_3_12.0,Product_Category_3_13.0,Product_Category_3_14.0,Product_Category_3_15.0,Product_Category_3_16.0,Product_Category_3_17.0,Product_Category_3_18.0
0,9051,765,13659,4995,0,12,38,2001,21064,10401.666667,1772,8890,5078.342105,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3521,2805,12816,3434,3009,11,97,1037,21510,9735.363636,7584,19689,15900.453608,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,9051,6561,13659,0,0,9,36,3456,13591,8431.222222,3456,8854,6685.611111,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19795,5353,2190,2411,0,55,28,1383,20261,7693.327273,3515,16462,10561.464286,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19795,2360,10312,4995,0,30,19,1486,16384,6644.400000,2012,10001,6112.526316,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,9051,6341,12816,4506,498,24,25,3111,19487,11575.625000,3887,19530,9781.720000,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
49996,19795,6341,13659,5889,1644,9,40,6182,15819,9175.555556,3431,8906,7291.825000,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
49997,10078,2805,13659,514,0,5,25,7012,15408,10946.800000,3430,8841,6746.040000,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49998,19795,2522,12816,0,0,45,2,926,23562,10242.511111,7866,12039,9952.500000,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Se divide el conjunto en entrenamiento y prueba

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y_train, test_size=0.20, random_state=42, shuffle=True)

Se aplica el clasificador

In [ ]:
pred_y1 = XGB(X_train, y_train, X_val)

sin tratamiento de outliers

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(pred_y1, y_val,squared=False)
print(rmse)

2355.9664381011044


con tratamiento de outliers

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(pred_y1, y_val,squared=False)
print(rmse)

2314.8470510812886


In [ ]:
y_val

array([ 2152,  8060, 15583, ...,  6087,  9992, 12401])

In [ ]:
pred_y1

array([ 5180.41  ,  8338.548 , 12266.563 , ...,  5546.7056,  7608.988 ,
       13314.026 ], dtype=float32)

# Segundo Modelo 

In [ ]:
maindf = pd.read_csv('txn_train.csv')

In [ ]:
gender = {'F':0, 'M':1}
age = {'0-17':0, '18-25':1, '26-35':2, '36-45':3, '46-50':4, '51-55':5, '55+':6}
city = {'A':0, 'B':1, 'C':2}
staycity = {'0':0, '1':1, '2':2, '3':3, '4+':4}

maindf = pd.read_csv('txn_train.csv')

In [ ]:
maindf["Gender"] = maindf["Gender"].apply(lambda x: gender[x])

In [ ]:
maindf["Age"] = maindf["Age"].apply(lambda x: age[x])

In [ ]:
maindf["City_Category"] = maindf["City_Category"].apply(lambda x: city[x])

In [ ]:
maindf["Stay_In_Current_City_Years"] = maindf["Stay_In_Current_City_Years"].apply(lambda x: staycity[x])

In [ ]:
def outlier_removal(df):
  q75,q25 = np.percentile(df.loc[:,'Purchase'],[75,25])
  iqr = q75-q25
  min = q25 - (iqr*1.5)
  max = q75 + (iqr*1.5)
  df = df.drop(df[df.loc[:,'Purchase']<min].index)
  df = df.drop(df[df.loc[:,'Purchase']>max].index)
  return df

In [ ]:
maindf = outlier_removal(maindf)

In [ ]:
maindf.shape

(50000, 12)

- Se obtienen las totales de cada atributo
- El minimo, maximo y promedio, quartiles de cada usuario, tipo poder adquisitivo
- El minimo, maximo y promedio, quartiles de cada producto, tipo el costo 

- lo mismo pero con las tres categorias de los productos

In [ ]:
maindf["Agesum"] =  getSumofAtr(maindf, maindf, "Age")

{0: 1379, 1: 9051, 2: 19795, 3: 10078, 4: 4262, 5: 3521, 6: 1914}
[9051, 3521, 9051, 19795, 19795, 10078, 10078, 4262, 9051, 19795, 3521, 9051, 19795, 4262, 10078, 19795, 19795, 3521, 9051, 10078, 19795, 9051, 19795, 9051, 10078, 19795, 19795, 1914, 10078, 19795, 19795, 10078, 19795, 10078, 19795, 4262, 3521, 19795, 19795, 9051, 9051, 9051, 9051, 19795, 4262, 9051, 9051, 1379, 19795, 9051, 9051, 10078, 19795, 9051, 19795, 19795, 4262, 10078, 19795, 10078, 3521, 19795, 3521, 19795, 19795, 19795, 3521, 9051, 19795, 10078, 19795, 9051, 19795, 10078, 19795, 19795, 4262, 9051, 19795, 19795, 19795, 9051, 9051, 19795, 19795, 1914, 19795, 10078, 9051, 9051, 10078, 9051, 19795, 4262, 10078, 9051, 10078, 10078, 10078, 10078, 3521, 3521, 9051, 3521, 9051, 19795, 1914, 4262, 4262, 19795, 10078, 19795, 19795, 19795, 19795, 19795, 19795, 19795, 19795, 4262, 10078, 10078, 9051, 9051, 19795, 9051, 10078, 19795, 3521, 4262, 19795, 9051, 19795, 1914, 10078, 19795, 19795, 19795, 9051, 19795, 9051, 9051, 

In [ ]:
maindf["Occupationsum"] = getSumofAtr(maindf, maindf, "Occupation")

{0: 6341, 1: 4175, 2: 2398, 3: 1526, 4: 6561, 5: 1061, 6: 1907, 7: 5353, 8: 152, 9: 619, 10: 1186, 11: 1067, 12: 2805, 13: 685, 14: 2522, 15: 1139, 16: 2360, 17: 3697, 18: 600, 19: 765, 20: 3081}
[765, 2805, 6561, 5353, 2360, 4175, 6341, 3697, 6561, 3081, 4175, 3697, 2360, 3697, 6341, 2398, 1139, 4175, 6561, 2805, 2805, 6561, 1061, 6561, 6341, 2398, 6341, 1526, 5353, 2522, 4175, 1067, 1526, 2805, 1526, 1907, 6341, 1139, 6341, 6341, 6561, 2360, 6561, 6341, 5353, 6561, 6561, 1186, 765, 5353, 6561, 2360, 1526, 6561, 1907, 3081, 6341, 5353, 2805, 5353, 3697, 5353, 4175, 1907, 3081, 2805, 4175, 6561, 765, 4175, 5353, 1907, 4175, 2805, 2360, 2398, 1067, 6561, 6561, 5353, 2398, 6341, 6561, 4175, 5353, 685, 3081, 4175, 765, 6341, 2398, 6561, 5353, 6341, 1526, 6561, 619, 6341, 4175, 4175, 4175, 4175, 6561, 1067, 2522, 765, 5353, 4175, 4175, 3697, 5353, 6341, 6341, 6341, 5353, 3697, 2360, 3697, 3697, 4175, 5353, 4175, 6561, 6561, 6561, 6561, 2360, 4175, 3081, 6561, 2805, 600, 6561, 1526, 6341, 5

In [ ]:
maindf["Product_Category_1sum"] = getSumofAtr(maindf, maindf, "Product_Category_1")

{1: 12816, 2: 2190, 3: 1852, 4: 1082, 5: 13659, 6: 1965, 7: 331, 8: 10312, 9: 30, 10: 465, 11: 2186, 12: 352, 13: 491, 14: 139, 15: 591, 16: 876, 17: 55, 18: 264, 19: 145, 20: 199}
[13659, 12816, 13659, 2190, 10312, 10312, 1082, 12816, 12816, 1965, 10312, 1965, 2190, 12816, 12816, 12816, 1852, 13659, 12816, 13659, 1965, 1965, 13659, 13659, 13659, 10312, 13659, 145, 13659, 1852, 12816, 10312, 12816, 12816, 13659, 12816, 13659, 1965, 13659, 12816, 10312, 1082, 2190, 1965, 591, 1965, 12816, 1852, 12816, 13659, 1965, 12816, 591, 12816, 13659, 1082, 1965, 13659, 13659, 10312, 13659, 13659, 1965, 12816, 1852, 491, 13659, 12816, 491, 1852, 10312, 1852, 13659, 2186, 352, 12816, 13659, 13659, 10312, 12816, 10312, 12816, 12816, 12816, 12816, 1965, 10312, 10312, 10312, 491, 13659, 10312, 12816, 13659, 13659, 13659, 1965, 2186, 2190, 13659, 12816, 10312, 12816, 12816, 876, 10312, 10312, 12816, 264, 13659, 12816, 1082, 12816, 352, 2186, 465, 13659, 12816, 264, 10312, 10312, 12816, 199, 10312, 13659

In [ ]:
maindf["ProductCategory_2sum"] = getSumofAtr(maindf, maindf, "Product_Category_2")

{2.0: 4506, 3.0: 292, 4.0: 2339, 5.0: 2411, 6.0: 1523, 7.0: 56, 8.0: 5889, 9.0: 503, 10.0: 299, 11.0: 1209, 12.0: 514, 13.0: 955, 14.0: 4995, 15.0: 3434, 16.0: 3993, 17.0: 1201, 18.0: 224}
[4995, 3434, 0, 2411, 4995, 4995, 2411, 3434, 4995, 3993, 955, 3993, 2339, 2411, 1523, 4506, 2339, 1209, 1523, 4995, 0, 5889, 1209, 0, 0, 4995, 5889, 0, 0, 2339, 4506, 0, 2411, 3434, 4995, 3993, 4995, 5889, 4995, 224, 4995, 2411, 2339, 3993, 0, 5889, 0, 2339, 4506, 5889, 299, 4995, 3993, 4506, 5889, 2411, 5889, 5889, 0, 0, 0, 0, 5889, 5889, 2339, 0, 0, 3993, 3993, 2339, 514, 2339, 5889, 0, 4995, 3434, 4995, 5889, 0, 1523, 4995, 3993, 4506, 1209, 4506, 5889, 0, 0, 4995, 3993, 514, 0, 5889, 0, 0, 3993, 3993, 3434, 0, 0, 4995, 4995, 3434, 0, 0, 4995, 1201, 2411, 0, 5889, 4506, 2411, 3434, 4995, 4995, 955, 5889, 3434, 0, 0, 0, 4506, 0, 3993, 5889, 0, 0, 5889, 0, 2339, 4506, 503, 2411, 1201, 0, 0, 1201, 3434, 4995, 3993, 0, 955, 0, 4506, 0, 5889, 955, 0, 4506, 1201, 4995, 955, 5889, 4506, 5889, 3434, 0, 0

In [ ]:
maindf["ProductCategory_3sum"] = getSumofAtr(maindf, maindf, "Product_Category_3")

{3.0: 51, 4.0: 159, 5.0: 1552, 6.0: 461, 8.0: 1156, 9.0: 1091, 10.0: 182, 11.0: 142, 12.0: 836, 13.0: 498, 14.0: 1644, 15.0: 2547, 16.0: 3009, 17.0: 1515, 18.0: 397}
[0, 3009, 0, 0, 0, 0, 397, 0, 3009, 0, 0, 0, 1091, 836, 1644, 1091, 836, 0, 3009, 0, 0, 0, 0, 0, 0, 1515, 0, 0, 0, 1552, 2547, 0, 0, 0, 0, 0, 1515, 0, 0, 0, 0, 0, 1552, 0, 0, 3009, 0, 0, 0, 0, 498, 0, 0, 1091, 0, 1156, 3009, 1644, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1552, 0, 836, 0, 0, 0, 1515, 0, 0, 0, 3009, 0, 0, 2547, 2547, 1552, 3009, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3009, 0, 0, 0, 0, 0, 0, 397, 0, 0, 2547, 0, 0, 0, 0, 0, 0, 3009, 0, 0, 0, 0, 0, 0, 1644, 0, 0, 1644, 0, 0, 2547, 1644, 836, 0, 0, 0, 0, 0, 1515, 0, 0, 1644, 0, 1552, 0, 1515, 3009, 0, 2547, 0, 0, 0, 0, 1552, 0, 1515, 0, 0, 1552, 397, 0, 0, 0, 0, 0, 0, 0, 1552, 3009, 0, 498, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 461, 2547, 0, 0, 0, 3009, 0, 1644, 0, 182, 0, 3009, 0, 0, 0, 2547, 0, 0, 0, 0, 0, 1644, 0, 1552, 0, 0, 0, 0, 1552, 836, 2547, 0, 1552, 0, 1552, 0, 15

In [ ]:
maindf["UserIDsum"] = getSumofAtr(maindf, maindf, "User_ID")

{1000001: 2, 1000002: 4, 1000003: 4, 1000005: 16, 1000006: 2, 1000007: 1, 1000008: 7, 1000009: 3, 1000010: 22, 1000011: 5, 1000012: 1, 1000013: 5, 1000014: 1, 1000015: 5, 1000016: 3, 1000017: 8, 1000018: 13, 1000019: 9, 1000020: 1, 1000021: 1, 1000022: 20, 1000023: 12, 1000024: 10, 1000025: 1, 1000026: 23, 1000027: 5, 1000028: 7, 1000029: 4, 1000030: 5, 1000031: 5, 1000032: 1, 1000033: 20, 1000034: 16, 1000035: 13, 1000036: 20, 1000037: 6, 1000039: 4, 1000041: 2, 1000042: 9, 1000043: 2, 1000044: 17, 1000045: 19, 1000046: 4, 1000047: 1, 1000048: 38, 1000049: 2, 1000051: 1, 1000053: 38, 1000054: 2, 1000055: 2, 1000056: 4, 1000057: 3, 1000058: 25, 1000059: 18, 1000060: 3, 1000061: 1, 1000062: 23, 1000063: 6, 1000064: 1, 1000065: 5, 1000066: 1, 1000067: 2, 1000068: 3, 1000069: 2, 1000070: 6, 1000071: 3, 1000072: 2, 1000073: 7, 1000074: 3, 1000075: 15, 1000076: 6, 1000077: 1, 1000078: 6, 1000079: 1, 1000081: 2, 1000082: 5, 1000083: 1, 1000084: 2, 1000086: 2, 1000087: 4, 1000090: 17, 1000091

In [ ]:
maindf["ProductIDsum"] = getSumofAtr(maindf, maindf, "Product_ID")

{'P00000142': 100, 'P00000242': 44, 'P00000342': 17, 'P00000442': 8, 'P00000542': 14, 'P00000642': 47, 'P00000742': 22, 'P00000842': 1, 'P00000942': 9, 'P00001042': 53, 'P00001142': 62, 'P00001242': 11, 'P00001342': 2, 'P00001442': 8, 'P00001542': 8, 'P00001642': 31, 'P00001742': 28, 'P00001842': 13, 'P00001942': 22, 'P00002042': 6, 'P00002142': 68, 'P00002242': 20, 'P00002342': 6, 'P00002442': 19, 'P00002542': 43, 'P00002642': 7, 'P00002742': 6, 'P00002842': 14, 'P00002942': 12, 'P00003042': 6, 'P00003142': 9, 'P00003242': 81, 'P00003442': 77, 'P00003542': 1, 'P00003642': 52, 'P00003842': 2, 'P00003942': 66, 'P00004042': 1, 'P00004142': 10, 'P00004242': 13, 'P00004342': 8, 'P00004442': 9, 'P00004542': 19, 'P00004642': 6, 'P00004742': 67, 'P00004842': 23, 'P00004942': 2, 'P00005042': 97, 'P00005142': 19, 'P00005242': 1, 'P00005342': 1, 'P00005442': 2, 'P00005542': 1, 'P00005642': 4, 'P00005742': 23, 'P00005942': 15, 'P00006042': 4, 'P00006142': 16, 'P00006242': 14, 'P00006342': 3, 'P00

In [ ]:
def XGB2(X_train, y_train, X_test):
  hyperparams = {}
  hyperparams["objective"] = "reg:linear"
  hyperparams["eta"] = 0.03
  hyperparams["min_child_weight"] = 10
  hyperparams["subsample"] = 0.8
  hyperparams["colsample_bytree"] = 0.7
  hyperparams["silent"] = 1
  hyperparams["max_depth"] = 10
        #params["max_delta_step"]=2
  hyperparams["seed"] = 0
        #params['eval_metric'] = "auc"
  hyplist = list(hyperparams.items())
  epochs = 1100
  xgb_train = xgb.DMatrix(X_train, label=y_train)
  xgb_test = xgb.DMatrix(X_test)
  model = xgb.train(hyplist, xgb_train, epochs)
  pred_y = model.predict(xgb_test)
  return pred_y

In [ ]:
def getAcquireAtr(maindf,maindf2,atrname):
  groupdf = maindf2.groupby(atrname)
  mind = {}
  maxd = {}
  meand = {}
  twentyfived = {}
  seventyfived = {}
  for name, cat in groupdf:
    mind[name] = min(np.array(cat["Purchase"]))
    maxd[name] = max(np.array(cat["Purchase"]))
    meand[name] = np.mean(np.array(cat["Purchase"]))
    twentyfived[name] = np.percentile(np.array(cat["Purchase"]),25)
    seventyfived[name] = np.percentile(np.array(cat["Purchase"]),75)
  
  minl = []
  maxl = []
  meanl = []
  twentyfivel = []
  seventyfivel = []
  for i, r in maindf.iterrows():
    name = r[atrname]
    minl.append(mind.get(name,0))
    maxl.append(maxd.get(name,0))
    meanl.append(meand.get(name,0))
    twentyfivel.append( twentyfived.get(name,0))
    seventyfivel.append( seventyfived.get(name,0))

  return minl,maxl,meanl,twentyfivel, seventyfivel

In [ ]:
min_price, max_price, mean_price, twentyfive_price, seventyfive_price = getAcquireAtr(maindf, maindf, "User_ID")

In [ ]:
maindf["UserID25Perc"] = twentyfive_price
maindf["UserID75Perc"] = seventyfive_price

In [ ]:
min_price_, max_price, mean_price, twentyfive_pricet, seventyfive_price = getAcquireAtr(maindf, maindf, "Product_ID")

In [ ]:
maindf["ProductID25Perc"] = twentyfive_price
maindf["ProductID75Perc"] = seventyfive_price

In [ ]:
min_price, max_price, mean_price, twentyfive_price, seventyfive_price = getAcquireAtr(maindf, maindf, "Product_Category_1")
maindf["Product_Cat1Min"] = min_price
maindf["Product_Cat1Max"] = max_price
maindf["Product_Cat1Mean"] = mean_price
maindf["Product_Cat1_25Per"] = twentyfive_price
maindf["Product_Cat1_75Per"] = seventyfive_price

In [ ]:
min_price, max_price, mean_price, twentyfive_price, seventyfive_price =  getAcquireAtr(maindf, maindf, "Product_Category_2")
maindf["Product_Cat2Min"] = min_price
maindf["Product_Cat2Max"] = max_price
maindf["Product_Cat2Mean"] = mean_price
maindf["Product_Cat2_25Perc"] = twentyfive_price
maindf["Product_Cat2_75Perc"] = seventyfive_price
min_price, max_price, mean_price, twentyfive_price, seventyfive_price = getAcquireAtr(maindf, maindf, "Product_Category_3")
maindf["Product_Cat3Min"] = min_price
maindf["Product_Cat3Max"] = max_price
maindf["Product_Cat3Mean"] = mean_price
maindf["Product_Cat3_25Perc"] = twentyfive_price
maindf["Product_Cat3_75Perc"] = seventyfive_price

Se obtiene la codificacion onehot para todas las variables categoricas

In [ ]:
categorical_columns = ["Gender","Age", "Occupation", "City_Category", "Stay_In_Current_City_Years",
                       "Marital_Status", "Product_Category_1", "Product_Category_2", "Product_Category_3"]

In [ ]:
maindf = pd.get_dummies(maindf, columns= categorical_columns)

In [ ]:
y_train = np.array(maindf["Purchase"])

In [ ]:
maindf = maindf.drop(columns =['User_ID','Product_ID','Purchase'])

In [ ]:
X_train = np.array(maindf).astype('float')

Se escalan a -1 a 1 

In [ ]:
Scaler = StandardScaler()
Scaler.fit(X_train)
X_train= Scaler.transform(X_train)

In [ ]:
X_train

array([[-0.48189829, -1.53506459,  0.82858191, ..., -0.25304849,
        -0.17676758, -0.08946255],
       [-1.32041299, -0.5086915 ,  0.65887715, ...,  3.95181176,
        -0.17676758, -0.08946255],
       [-0.48189829,  1.38104249,  0.82858191, ..., -0.25304849,
        -0.17676758, -0.08946255],
       ...,
       [-0.32617413, -0.5086915 ,  0.82858191, ..., -0.25304849,
        -0.17676758, -0.08946255],
       [ 1.14721599, -0.65107561,  0.65887715, ..., -0.25304849,
        -0.17676758, -0.08946255],
       [-0.48189829, -0.05990483,  0.65887715, ...,  3.95181176,
        -0.17676758, -0.08946255]])

In [ ]:
maindf

,Agesum,Occupationsum,Product_Category_1sum,ProductCategory_2sum,ProductCategory_3sum,UserIDsum,ProductIDsum,UserID25Perc,UserID75Perc,ProductID25Perc,ProductID75Perc,Product_Cat1Min,Product_Cat1Max,Product_Cat1Mean,Product_Cat1_25Per,Product_Cat1_75Per,Product_Cat2Min,Product_Cat2Max,Product_Cat2Mean,Product_Cat2_25Perc,Product_Cat2_75Perc,Product_Cat3Min,Product_Cat3Max,Product_Cat3Mean,Product_Cat3_25Perc,Product_Cat3_75Perc,Gender_0,Gender_1,Age_0,Age_1,Age_2,Age_3,Age_4,Age_5,Age_6,Occupation_0,Occupation_1,Occupation_2,Occupation_3,Occupation_4,...,Product_Category_1_13,Product_Category_1_14,Product_Category_1_15,Product_Category_1_16,Product_Category_1_17,Product_Category_1_18,Product_Category_1_19,Product_Category_1_20,Product_Category_2_2.0,Product_Category_2_3.0,Product_Category_2_4.0,Product_Category_2_5.0,Product_Category_2_6.0,Product_Category_2_7.0,Product_Category_2_8.0,Product_Category_2_9.0,Product_Category_2_10.0,Product_Category_2_11.0,Product_Category_2_12.0,Product_Category_2_13.0,Product_Category_2_14.0,Product_Category_2_15.0,Product_Category_2_16.0,Product_Category_2_17.0,Product_Category_2_18.0,Product_Category_3_3.0,Product_Category_3_4.0,Product_Category_3_5.0,Product_Category_3_6.0,Product_Category_3_8.0,Product_Category_3_9.0,Product_Category_3_10.0,Product_Category_3_11.0,Product_Category_3_12.0,Product_Category_3_13.0,Product_Category_3_14.0,Product_Category_3_15.0,Product_Category_3_16.0,Product_Category_3_17.0,Product_Category_3_18.0
0,9051,765,13659,4995,0,12,38,6567.25,12784.0,6567.25,6531.00,1713,8907,6239.254411,5237.00,7160.00,197,23732,7113.648849,5342.00,8608.50,0,0,0.000000,0.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3521,2805,12816,3434,3009,11,97,5353.00,13762.5,5353.00,19184.00,3792,19708,13626.528636,11544.00,15813.25,214,23961,10272.755970,5981.00,15355.00,197,23942,12002.242273,7759.0,15841.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,9051,6561,13659,0,0,9,36,4535.00,11572.0,4535.00,7541.00,1713,8907,6239.254411,5237.00,7160.00,0,0,0.000000,0.00,0.00,0,0,0.000000,0.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19795,5353,2190,2411,0,55,28,4009.50,11511.0,4009.50,13179.25,3176,16504,11293.831050,9669.75,13212.75,691,19708,9158.684778,2843.50,13614.50,0,0,0.000000,0.0,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19795,2360,10312,4995,0,30,19,3704.25,7897.5,3704.25,7894.00,1940,10082,7507.526862,6041.75,9721.00,197,23732,7113.648849,5342.00,8608.50,0,0,0.000000,0.0,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,9051,6341,12816,4506,498,24,25,7113.00,15666.0,7113.00,11846.00,3792,19708,13626.528636,11544.00,15813.25,3797,19708,13733.653795,11566.00,15831.75,774,20688,13440.626506,8835.0,16483.0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
49996,19795,6341,13659,5889,1644,9,40,6880.00,8827.0,6880.00,8709.25,1713,8907,6239.254411,5237.00,7160.00,698,20686,10352.484972,6879.00,15569.00,1719,20597,10222.063260,6863.0,15320.5,1,0,0,0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
49997,10078,2805,13659,514,0,5,25,7052.00,15357.0,7052.00,7163.00,1713,8907,6239.254411,5237.00,7160.00,758,20354,7071.340467,5317.25,8679.00,0,0,0.000000,0.0,0.0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0

Se divide el dataset en Entrenamiento y Prueba

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42, shuffle=True)

Se aplica el regresor

In [ ]:
predy = XGB2(X_train, y_train, X_val)

sin tratamiento de outlieres

In [ ]:
rmse = mean_squared_error(predy, y_val,squared=False)
print(rmse)

2519.4695625516074


con tratamiento de outliers

In [ ]:
rmse = mean_squared_error(predy, y_val,squared=False)
print(rmse)

2459.4671281188953


# Ensamble de modelos

In [ ]:
finalpred = 0.7*pred_y1 + 0.3*predy

Sin tratamiento de outliers

In [ ]:
rmse = mean_squared_error(finalpred, y_val,squared=False)
print(rmse)

2354.771319147697


Con tratamiento de outliers

In [ ]:
rmse = mean_squared_error(finalpred, y_val,squared=False)
print(rmse)

2305.129010237747


# Resultados finales

| Model / Error | No outlier treatment | outlier treatment  | other preprocessing|
| :- | -: | :-: ||
| Regresion lineal |  2513.015 | 2385.9293 | 2510.34
| Random Forest | 2568.7959 | 2417.9844 | 2381.20
| Arboles de Decision | 2521.6132 | 2389.5480 | 2431.78
| AdaBoost | 2658.1648 | 2529.4123 | 2646.21
| Gradient Boosting | 2514.2014 | 2514.2014 | 2373.10
| XGBRegressor | 2480.219 | 2342.9594 | 2310.68
| Regresion Rigida | 2512.6481 | 2385.8774 | 2385.87
| Bayes Ingenuo | 4560.0224 | 4296.7736 | 4056
| SVM | - | - | -
| Redes Neuronales Profundas |  4694.6738 | - | -
| Regresor LGBM | 2555.3845 | 2407.548 | 2366
| Ensamble de dos XGB | 2354.7713 | 2305.12 |